# Step 1: Load the meta-llama/Llama-3.2-3B Model and Tokenizer

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,  # if using bitsandbytes
    device_map="auto",
    trust_remote_code=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Step 2: Prepare the LoRA Configuration with PEFT

In [2]:
from peft import PeftModel, get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Only apply LoRA if not already applied
if not isinstance(model, PeftModel):
    model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


# Step 3: Load and Preprocess Dataset¶

## zh-en 

In [3]:
from datasets import load_dataset

# Load full dataset (zh-en)
dataset_zh_en = load_dataset("wmt19", "zh-en")

In [4]:
print(len(dataset_zh_en["train"]))
print(len(dataset_zh_en["validation"]))

25984574
3981


In [5]:
# Set slice sizes
TRAIN_SIZE = 70_000
VAL_SIZE = 1000

# Randomly shuffle and select subsets
small_dataset_zh_en = {
    "train": dataset_zh_en["train"].shuffle(seed=42).select(range(TRAIN_SIZE)),
    "validation": dataset_zh_en["validation"].shuffle(seed=42).select(range(VAL_SIZE))
}

In [6]:
print(len(small_dataset_zh_en["train"]))
print(len(small_dataset_zh_en["validation"]))

70000
1000


## ne-en

In [7]:
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict

# 1. Load full dataset (only has a "train" split)
dataset_ne_en = load_dataset("iamTangsang/Nepali-to-English-Translation-Dataset")

In [8]:
print(len(dataset_ne_en["train"]))
print(len(dataset_ne_en["validation"]))

702697
10866


In [9]:
# Set slice sizes
TRAIN_SIZE = 70_000
VAL_SIZE = 1000

# Randomly shuffle and select subsets
small_dataset_ne_en = {
    "train": dataset_ne_en["train"].shuffle(seed=42).select(range(TRAIN_SIZE)),
    "validation": dataset_ne_en["validation"].shuffle(seed=42).select(range(VAL_SIZE))
}

In [10]:
print(len(small_dataset_ne_en["train"]))
print(len(small_dataset_ne_en["validation"]))

70000
1000


## combine them

In [11]:
# zh-en format
zh_en_train = small_dataset_zh_en["train"].map(lambda x: {
    "input": x["translation"]["zh"].strip(),
    "output": x["translation"]["en"].strip(),
    "lang_pair": "zh-en"
})

zh_en_val = small_dataset_zh_en["validation"].map(lambda x: {
    "input": x["translation"]["zh"].strip(),
    "output": x["translation"]["en"].strip(),
    "lang_pair": "zh-en"
})

# ne-en format
ne_en_train = small_dataset_ne_en["train"].map(lambda x: {
    "input": x["source"].strip(),
    "output": x["target"].strip(),
    "lang_pair": "ne-en"
})

ne_en_val = small_dataset_ne_en["validation"].map(lambda x: {
    "input": x["source"].strip(),
    "output": x["target"].strip(),
    "lang_pair": "ne-en"
})

In [12]:
print(zh_en_train)
print(ne_en_train)

Dataset({
    features: ['translation', 'input', 'output', 'lang_pair'],
    num_rows: 70000
})
Dataset({
    features: ['source', 'target', 'input', 'output', 'lang_pair'],
    num_rows: 70000
})


In [13]:
print(zh_en_train.features)
print(ne_en_train.features)

{'translation': Translation(languages=['zh', 'en'], id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None), 'lang_pair': Value(dtype='string', id=None)}
{'source': Value(dtype='string', id=None), 'target': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None), 'lang_pair': Value(dtype='string', id=None)}


In [14]:
from datasets import concatenate_datasets

combined_train = concatenate_datasets([zh_en_train, ne_en_train]).shuffle(seed=42)
combined_val = concatenate_datasets([zh_en_val, ne_en_val]).shuffle(seed=42)

In [15]:
print(combined_train)
print(combined_val)

Dataset({
    features: ['translation', 'input', 'output', 'lang_pair', 'source', 'target'],
    num_rows: 140000
})
Dataset({
    features: ['translation', 'input', 'output', 'lang_pair', 'source', 'target'],
    num_rows: 2000
})


In [16]:
zh2en_templates = [
    "User: Translate Chinese to English: {input}\nAssistant: {output}",
    "User: What is the English translation of: {input}?\nAssistant: {output}",
    "User: Please convert this to English: {input}\nAssistant: {output}"
]

en2zh_templates = [
    "User: Translate English to Chinese: {input}\nAssistant: {output}",
    "User: What is the Chinese translation of: {input}?\nAssistant: {output}",
    "User: Please convert this to Chinese: {input}\nAssistant: {output}"
]

ne2en_templates = [
    "User: Translate Nepali to English: {input}\nAssistant: {output}",
    "User: What is the English translation of: {input}?\nAssistant: {output}",
    "User: Please convert this to English: {input}\nAssistant: {output}"
]

en2ne_templates = [
    "User: Translate English to Nepali: {input}\nAssistant: {output}",
    "User: What is the Nepali translation of: {input}?\nAssistant: {output}",
    "User: Please convert this to Nepali: {input}\nAssistant: {output}"
]

In [17]:
import random
def preprocess(example,batched=True):
    lang = example["lang_pair"]
    prompt = ""

    if lang == "zh-en":
        if random.random() < 0.5:
            prompt = random.choice(zh2en_templates).format(input=example["input"], output=example["output"])
        else:
            prompt = random.choice(en2zh_templates).format(input=example["output"], output=example["input"])

    elif lang == "ne-en":
        if random.random() < 0.5:
            prompt = random.choice(ne2en_templates).format(input=example["input"], output=example["output"])
        else:
            prompt = random.choice(en2ne_templates).format(input=example["output"], output=example["input"])

    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

In [18]:
tokenized_dataset = {
    "train": combined_train.map(preprocess, remove_columns=combined_train.column_names),
    "validation": combined_val.map(preprocess, remove_columns=combined_val.column_names)
}

In [19]:
print(tokenized_dataset)

{'train': Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 140000
}), 'validation': Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})}


# Step 4: Setup Training with Trainer

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    # for checkpoint
    output_dir="./Llama-3.2-3B/zh-en-ne",
    save_steps=250,
    save_total_limit=4,
    # for logging, used by tensorboard
    logging_dir="./Llama-3.2-3B/logging", 
    logging_steps=100,
    report_to="tensorboard", #report_to="tensorboard" or "wandb",
    # # if report to tensor board then run: 
    # # tensorboard --logdir=~/translation_project/Llama-3.2-3B/logging/ --port=6006
    # for training
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    lr_scheduler_type="cosine", # or "linear"(default) or "constant_with_warmup",
    warmup_steps=200,
    weight_decay=0.01,
    fp16=True,
    num_train_epochs=1,
    max_steps=10000,
    # for eval
    eval_strategy="steps",
    eval_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# Step 5: Evaluate before training

In [36]:
from evaluate import load
from tqdm import tqdm
import torch

# Load metrics
bleu = load("bleu")
chrf = load("chrf")  # We use this to compute chrF++, setting word_order=2 later.

def evaluate_direction(model, tokenizer, dataset, direction="zh2en", max_samples=100, show_samples=3):
    """
    Evaluate a translation model on a given direction by computing both BLEU and chrF++ scores.
    The scores are normalized for readability (BLEU is reported as a percentage).
    
    Parameters:
      - model: The translation model.
      - tokenizer: The tokenizer corresponding to the model.
      - dataset: A dataset of translation examples. Each example should include:
          * "lang_pair": e.g. "zh-en" or "ne-en".
          * "input": The source text.
          * "output": The target translation.
      - direction: One of "zh2en", "en2zh", "ne2en", or "en2ne".
      - max_samples: Maximum number of samples from the dataset to evaluate.
      - show_samples: Number of sample predictions (with prompt and reference) to print.
    """
    predictions = []
    references_bleu = []  # For BLEU, each reference is a list (even if single-reference)
    references_chrf = []  # For chrF++, each reference is a simple string
    shown = 0

    print(f"\n🔍 Evaluating direction: {direction} | Max samples: {max_samples}\n")

    # Loop over the first max_samples examples in the dataset
    for ex in tqdm(dataset.select(range(max_samples)), desc=f"Eval: {direction}"):
        lang = ex["lang_pair"]

        # Determine prompt and reference according to the translation direction.
        # If using en2zh or en2ne, we swap the roles of input and output.
        if direction == "zh2en" and lang == "zh-en":
            prompt = f"User: Translate Chinese to English: {ex['input']}\nAssistant:"
            ref = ex["output"]
        elif direction == "en2zh" and lang == "zh-en":
            prompt = f"User: Translate English to Chinese: {ex['output']}\nAssistant:"
            ref = ex["input"]
        elif direction == "ne2en" and lang == "ne-en":
            prompt = f"User: Translate Nepali to English: {ex['input']}\nAssistant:"
            ref = ex["output"]
        elif direction == "en2ne" and lang == "ne-en":
            prompt = f"User: Translate English to Nepali: {ex['output']}\nAssistant:"
            ref = ex["input"]
        else:
            continue  # Skip if the language pair doesn't match the required translation direction.

        # Tokenize the prompt and ensure tensors are on the model's device.
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)

        # Post-process the output by splitting at "Assistant:" and stripping extra whitespace.
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()

        # Optionally print a few sample outputs for visual inspection.
        if shown < show_samples:
            shown += 1
            print(f"\n🔹 Sample #{shown}")
            print("📥 Prompt:", prompt)
            print("🟢 Prediction:", pred)
            print("🔸 Reference:", ref)

        predictions.append(pred)
        references_bleu.append([ref])  # BLEU expects each reference as a list.
        references_chrf.append(ref)

    # Compute BLEU and convert to a percentage for readability.
    bleu_result = bleu.compute(predictions=predictions, references=references_bleu)
    bleu_score = bleu_result["bleu"] * 100  # Normalization to percentage

    # Compute chrF++ with word_order=2 (the setting for chrF++).
    chrf_result = chrf.compute(predictions=predictions, references=references_chrf, word_order=2)
    chrf_score = chrf_result["score"]
    
    # Print results in a normalized, popular format.
    print(f"\n✅ {direction.upper()} BLEU Score: {bleu_score:.2f}")
    print(f"✅ {direction.upper()} chrF++ Score: {chrf_score:.2f}")

In [27]:
# Call for each direction
evaluate_direction(model, tokenizer, combined_val, "zh2en",max_samples=100, show_samples=3)
evaluate_direction(model, tokenizer, combined_val, "en2zh",max_samples=100, show_samples=3)
evaluate_direction(model, tokenizer, combined_val, "ne2en",max_samples=100, show_samples=3)
evaluate_direction(model, tokenizer, combined_val, "en2ne",max_samples=100, show_samples=3)


🔍 Evaluating direction: zh2en | Max samples: 100



Eval: zh2en:   2%|█▉                                                                                            | 2/100 [00:08<06:34,  4.02s/it]


🔹 Sample #1
📥 Prompt: User: Translate Chinese to English: 布鲁克斯 71 岁，是一位佐治亚州前议员和终身维权活动家。
Assistant:
🟢 Prediction: 好了，布鲁克斯先生，您的英文是这样的：
Brooks, a 71-year-old former Georgia state legislator and lifelong civil rights activist, has been arrested for the second time in less than a week for protesting in front of the state Capitol. He was released on a $1,000 bond after his arrest Tuesday.
Brooks has been protesting in front
🔸 Reference: Brooks is a 71-year-old former Georgia state congressman and lifelong civil rights activist.


Eval: zh2en:   4%|███▊                                                                                          | 4/100 [00:16<06:30,  4.06s/it]


🔹 Sample #2
📥 Prompt: User: Translate Chinese to English: 国家邮政局2016年曾发布报告，指出2015年“双11”期间（11月11日至16日）的快件量约7．8亿件，使用超过30亿条编织袋，99.22亿个包装箱，169.85亿米胶带，胶带长度可以绕地球赤道425圈。
Assistant:
🟢 Prediction: Okay. Here's your translation: "The National Post Office released a report in 2016, stating that in
🔸 Reference: In 2016, the State Post Bureau published a report stating that approximately 780 million items were delivered by express couriers during the 2015 “Singles’ Day” period (November 11-16), with more than 3 billion woven bags, 9.922 billion packaging cartons, and 16.985 billion meters of adhesive tape used. The length of adhesive tape used could go round the equator 425 times.


...


🔹 Sample #2
📥 Prompt: User: Translate English to Nepali: You can change it at any other time.
Assistant:
🟢 Prediction: Sure. I can
🔸 Reference: तपाईं यसलाई कुनै पनि समयमा परिवर्तन गर्न सक्नुहुनेछ।


Eval: en2ne:   9%|████████▍                                                                                     | 9/100 [00:24<03:23,  2.23s/it]


🔹 Sample #3
📥 Prompt: User: Translate English to Nepali: I was the head.
Assistant:
🟢 Prediction: I was
🔸 Reference: म त्यसको हेड थिएँ ।


Eval: en2ne: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:15<00:00,  3.76s/it]


✅ EN2NE BLEU Score: 0.00
✅ EN2NE chrF++ Score: 0.12


# Step 6: Train the Model

In [33]:
import torch
from torch.serialization import add_safe_globals
import numpy
torch.cuda.empty_cache()

# Add all necessary globals to the safe allowlist
add_safe_globals([
    numpy.core.multiarray._reconstruct,
    numpy.ndarray,
    numpy.dtype,
    numpy.dtypes.UInt32DType  # Newly added to allow UInt32DType
])

trainer.train(resume_from_checkpoint=True)

/home/jliu16@cfreg.local/downloads/envs/env0/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss,Validation Loss
5500,0.418100,0.479148
6000,0.424300,0.479318
6500,0.444300,0.477211
7000,0.423600,0.475966
7500,0.422100,0.475118
8000,0.413800,0.474945
8500,0.409400,0.475056


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

# Step 7: Evaluate afer training

In [49]:
# Call for each direction
evaluate_direction(model, tokenizer, combined_val, "zh2en",max_samples=200, show_samples=3)
evaluate_direction(model, tokenizer, combined_val, "en2zh",max_samples=200, show_samples=3)
evaluate_direction(model, tokenizer, combined_val, "ne2en",max_samples=200, show_samples=3)
evaluate_direction(model, tokenizer, combined_val, "en2ne",max_samples=200, show_samples=3)


🔍 Evaluating direction: zh2en | Max samples: 200



Eval: zh2en:   1%|▉                                                                                             | 2/200 [00:01<02:45,  1.19it/s]


🔹 Sample #1
📥 Prompt: User: Translate Chinese to English: 布鲁克斯 71 岁，是一位佐治亚州前议员和终身维权活动家。
Assistant:
🟢 Prediction: Brooks, 71, is a former Georgia state legislator and lifetime civil rights activist.
🔸 Reference: Brooks is a 71-year-old former Georgia state congressman and lifelong civil rights activist.


Eval: zh2en:   2%|█▉                                                                                            | 4/200 [00:07<06:47,  2.08s/it]


🔹 Sample #2
📥 Prompt: User: Translate Chinese to English: 国家邮政局2016年曾发布报告，指出2015年“双11”期间（11月11日至16日）的快件量约7．8亿件，使用超过30亿条编织袋，99.22亿个包装箱，169.85亿米胶带，胶带长度可以绕地球赤道425圈。
Assistant:
🟢 Prediction: The National Postal Administration of China reported in 2016 that the number of parcels sent during "Double 11" (11 November to 16 November) in 2015 was 780 million, using 30 million woven bags, 992 million boxes, 169.85 km of tape, and 425 times around the equator.
🔸 Reference: In 2016, the State Post Bureau published a report stating that approximately 780 million items were delivered by express couriers during the 2015 “Singles’ Day” period (November 11-16), with more than 3 billion woven bags, 9.922 billion packaging cartons, and 16.985 billion meters of adhesive tape used. The length of adhesive tape used could go round the equator 425 times.


...


🔹 Sample #2
📥 Prompt: User: Translate English to Nepali: You can change it at any other time.
Assistant:
🟢 Prediction: तपाईंले त्यो समयमा कुनै पनि समयमा पुन: परिवर्तन गर्न सक्नुहुन्छ।
🔸 Reference: तपाईं यसलाई कुनै पनि समयमा परिवर्तन गर्न सक्नुहुनेछ।


Eval: en2ne:   4%|████▏                                                                                         | 9/200 [00:05<01:29,  2.12it/s]


🔹 Sample #3
📥 Prompt: User: Translate English to Nepali: I was the head.
Assistant:
🟢 Prediction: म पनि सुरु भएँ ।
🔸 Reference: म त्यसको हेड थिएँ ।


Eval: en2ne: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [04:51<00:00,  1.46s/it]


✅ EN2NE BLEU Score: 4.33
✅ EN2NE chrF++ Score: 27.19


# Step 8: Inference

In [ ]:
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer

# base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B")
# tokenizer = AutoTokenizer.from_pretrained("jingmingliu01/Llama-3.2-3B-lora-zh-en-ne-8500steps")

# model = PeftModel.from_pretrained(base_model, "jingmingliu01/Llama-3.2-3B-lora-zh-en-ne-8500steps")

In [ ]:
# # IF load from local

# from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel

# base = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", load_in_4bit=True, device_map="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("Llama-3.2-3B-lora-zh-en-ne-local-8500steps", trust_remote_code=True)
# model = PeftModel.from_pretrained(base, "Llama-3.2-3B-lora-zh-en-ne-local-8500steps")

In [38]:
def simple_translate(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [43]:
prompt = "User: Translate English to Chinese: I am happly that you are here.\nAssistant:"
print(simple_translate(prompt))

User: Translate English to Chinese: I am happly that you are here.
Assistant: 我很高兴你来到这里.


In [44]:
prompt = "User: Translate Chinese to English: 爱是一颗幸福的子弹.\nAssistant:"
print(simple_translate(prompt))

User: Translate Chinese to English: 爱是一颗幸福的子弹.
Assistant: Love is a happy bullet.


In [45]:
prompt = "User: Translate English to Nepali: I am happly that you are here.\nAssistant:"
print(simple_translate(prompt))

User: Translate English to Nepali: I am happly that you are here.
Assistant: तपाईं यहाँ छन् भन्ने मलाई खुशी हुन्छ ।


In [47]:
prompt = "User: Translate Nepali to English: प्रेम खुशीको गोली हो।\nAssistant:"
print(simple_translate(prompt))

User: Translate Nepali to English: प्रेम खुशीको गोली हो।
Assistant: Love is a happy bullet.


# Save

In [34]:
model.push_to_hub("jingmingliu01/Llama-3.2-3B-lora-zh-en-ne-8500steps")
tokenizer.push_to_hub("jingmingliu01/Llama-3.2-3B-lora-zh-en-ne-8500steps")

adapter_model.safetensors:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jingmingliu01/Llama-3.2-3B-lora-zh-en-ne-8500steps/commit/45718c02ad32e1f06ba7c35552eaed140c0954a4', commit_message='Upload tokenizer', commit_description='', oid='45718c02ad32e1f06ba7c35552eaed140c0954a4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jingmingliu01/Llama-3.2-3B-lora-zh-en-ne-8500steps', endpoint='https://huggingface.co', repo_type='model', repo_id='jingmingliu01/Llama-3.2-3B-lora-zh-en-ne-8500steps'), pr_revision=None, pr_num=None)

In [35]:
model.save_pretrained("Llama-3.2-3B-lora-zh-en-ne-local-8500steps")
tokenizer.save_pretrained("Llama-3.2-3B-lora-zh-en-ne-local-8500steps")

('Llama-3.2-3B-lora-zh-en-ne-local-8500steps/tokenizer_config.json',
 'Llama-3.2-3B-lora-zh-en-ne-local-8500steps/special_tokens_map.json',
 'Llama-3.2-3B-lora-zh-en-ne-local-8500steps/tokenizer.json')

In [50]:
from safetensors.torch import load_file
import torch

# Load the two LoRA adapter files
w1 = load_file("/home/jliu16@cfreg.local/translation_project/Llama-3.2-3B-lora-zh-en-ne-local/adapter_model.safetensors")
w2 = load_file("/home/jliu16@cfreg.local/translation_project/Llama-3.2-3B-lora-zh-en-ne-local-8500steps/adapter_model.safetensors")

# Step 1: List keys
print("🔑 Available keys in adapter:")
for key in w1.keys():
    print(key)

# # Step 2: Pick a key to compare (use any from the printed list)
# key = list(w1.keys())[0]  # You can also manually set e.g.:
# # key = "base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight"

# Step 3: Compare the weights
    same = torch.allclose(w1[key], w2[key], atol=1e-6)
    print(f"\n🧪 Key: {key}")
    print("Weights identical?" , same)

🔑 Available keys in adapter:
base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight
base_model.model.model.layers.10.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.10.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.10.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.10.self_attn.v_proj.lora_B.weight
base_model.model.model.layers.11.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.11.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.11.self_attn.v_proj.lora_A.weight
base_model.model.mo